In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()

llm = init_chat_model("claude-sonnet-4-5-20250929")

In [ ]:
from langgraph.graph import StateGraph
from langchain_core.messages import AnyMessage, HumanMessage

In [ ]:
from typing import List, TypedDict

In [ ]:

# Define the State using TypedDict
class ChatbotStateTD(TypedDict):
    messages: List[AnyMessage]

In [ ]:
# Initial State
initial_state_typeddict = ChatbotStateTD(messages=[])
type(initial_state_typeddict)

In [ ]:

print(f"Initial State (TypedDict): {initial_state_typeddict}")

In [ ]:

# Initial State with dictionary
initial_state_typeddict = ChatbotStateTD({"messages": []})

print(f"Initial State (TypedDict): {initial_state_typeddict}")

In [ ]:
# Initial State with wrong types
initial_state_typeddict_1 = ChatbotStateTD({"messages": True})
initial_state_typeddict_2 = ChatbotStateTD({"messages": ["hello"]})

print(f"{initial_state_typeddict_1=}\n{initial_state_typeddict_2=}")

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
# Define the State using TypedDict
class ChatbotStateBM(BaseModel):
    messages: List[AnyMessage] = Field(default_factory=list)

In [ ]:
# Initial State
initial_state_pydantic = ChatbotStateBM()
initial_state_pydantic

In [ ]:
print(f"Initial State (Pydantic): {initial_state_pydantic}")
print(f"Initial State (Pydantic): {initial_state_pydantic.model_dump()}")

In [ ]:

# Initial State with non-default value
initial_state_pydantic = ChatbotStateBM(messages=[HumanMessage("Hello!")])
initial_state_pydantic

In [ ]:
# Initial State with wrong type
initial_state_pydantic = ChatbotStateBM(messages="Hello!")

In [ ]:
# Define graph with multiple states (input, internal, and output)
class InputState(TypedDict):
    user_input: str

class OutputState(TypedDict):
    graph_output: str

class InternalState(TypedDict):
    internal_var: str
    user_input: str
    graph_output: str

graph = StateGraph(InternalState, input_schema=InputState, output_schema=OutputState)

#### We can also define private input and output state node states (images)

In [ ]:
# Default reducer function
class State(TypedDict):
    foo: int
    bar: str
    baz: float

In [ ]:
# Cutom reducer function
from typing import Annotated
from operator import add

def custom_reducer(old_value, new_value) -> float:
  return (old_value + new_value)/2

def increment(old_value, new_value) -> int:
  return old_value + 1

class State(TypedDict):
    foo: Annotated[float, custom_reducer]
    baz: Annotated[int, increment] # Count nbr of updates
    bar: Annotated[list[str], add]

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

In [ ]:
# define state
class State(TypedDict):
    messages: Annotated[list, add_messages]

# define chatbot 1 node
def chatbot1(state: State) -> State:
    llm_response = "Hello from the chatbot1!"
    return {"messages": [llm_response]}

# define chatbot 2 node
def chatbot2(state: State) -> State:
    llm_response = "Hello from the chatbot2!"
    return {"messages": [llm_response]}

In [ ]:
# Instantiate a graph builder
graph_builder = StateGraph(State)

# Add node
graph_builder.add_node("chatbot1", chatbot1)

# We can also add caching to an expensive node
graph_builder.add_node("chatbot2", chatbot2)

In [ ]:

graph_builder.compile()

In [ ]:
from langgraph.graph import START, END

In [ ]:

# define normal (direct) edge
graph_builder.add_edge(START, "chatbot1")

In [ ]:
# define conditional edge
def routing_function(state: State) -> str:
    if len(state["messages"]) > 1000:
        return "chatbot2"
    return "chatbot1"

graph_builder.add_conditional_edges(START, routing_function)

In [ ]:

# define conditional edge with path mapping
def use_chatbot2(state: State) -> str:
    if len(state["messages"]) > 1000:
        return True
    return False

graph_builder.add_conditional_edges(START, use_chatbot2, {True: "chatbot2", False:"chatbot1"})

In [ ]:
from typing import Annotated, List
from pydantic import BaseModel

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_core.messages import AnyMessage

In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()

llm = init_chat_model("claude-sonnet-4-5-20250929")

In [ ]:
# define state
class MessageState(BaseModel):
    messages: Annotated[List[AnyMessage], add_messages]  # from langgraph.graph import MessagesState


# define chatbot node
def chatbot(state: MessageState):
    reponse = llm.invoke(state.messages)
    return {"messages": [reponse]}

# create graph
graph_builder = StateGraph(MessageState)

# add nodes
graph_builder.add_node("chatbot", chatbot)

# add edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# compile graph
graph = graph_builder.compile()

In [ ]:
# visualize graph
graph

In [ ]:

# run graph with invoke
final_state = graph.invoke({"messages":[{"role":"user",
                                         "content":"Hello!"
                                         }]
                            }) # can work with just ["hello!"] because we used predefined reducer

In [ ]:
final_state

In [ ]:
final_state["messages"][-1].content

In [ ]:
# run graph with stream
final_state = graph.stream({"messages":["hello!"]}, stream_mode="values") # Emit all values in the state after each step

In [ ]:
final_state

In [ ]:
list(final_state)

In [ ]:
import time

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        message.pretty_print()
        time.sleep(1)

In [ ]:
print_stream(final_state)

In [ ]:
from langchain_core.runnables import RunnableConfig
from langgraph.graph import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]

class ConfigSchema(TypedDict):
    llm_model: str

def chatbot(state: State, config:RunnableConfig) -> State:
    if config["configurable"]["llm_model"] == "gemini":
        llm_response = "Hello from the Gemini chatbot!"
    elif config["configurable"]["llm_model"] == "openai":
        llm_response = "Hello from the OpenAI chatbot!"
    else:
        raise(ValueError("Invalid/Unsupported LLM model."))
    return {"messages": [llm_response]}

builder = StateGraph(State, context_schema=ConfigSchema)

builder.add_node("chatbot", chatbot)

builder.set_entry_point("chatbot")
builder.set_finish_point("chatbot")

graph = builder.compile()

In [ ]:
graph

In [ ]:
config_with_gemini = {"configurable": {"llm_model": "gemini"}}
config_with_openai = {"configurable": {"llm_model": "openai"}}

final_state_1 = graph.invoke({"messages":["Hello!"]}, config_with_gemini)
final_state_2 = graph.invoke({"messages":["Hello!"]}, config_with_openai)

In [ ]:
final_state_1["messages"][-1].content, final_state_2["messages"][-1].content

In [ ]:
config_with_wrong_llm = {"configurable": {"llm_model": "grok"}}
final_state = graph.invoke({"messages":["Hello!"]}, config_with_wrong_llm)

In [ ]:
import random
from typing_extensions import Literal
from langgraph.types import Command

In [ ]:
# Define the Graph State
class CoinFlipState(TypedDict):
    result: str
    message: str

# Define the Nodes
def initialize_state(state: CoinFlipState):
    return {"result": "unset", "message": "Ready to flip the coin!"}

def flip_coin(state: CoinFlipState) -> Command[Literal["heads_action", "tails_action"]]:
    coin_outcome = random.choice(["Heads", "Tails"])

    if coin_outcome == "Heads":
        next_node = "heads_action"
        update_message = "It's Heads!"
    else:
        next_node = "tails_action"
        update_message = "It's Tails!"
    return Command(
        update={"result": coin_outcome, "message": update_message},
        goto=next_node,
    )

def heads_action(state: CoinFlipState):
    return {"message": state["message"] + " We got lucky!"}

def tails_action(state: CoinFlipState):
    return {"message": state["message"] + " Better luck next time."}

# Build the Graph
builder = StateGraph(CoinFlipState)

# Add nodes to the graph
builder.add_node("initialize_state", initialize_state)
builder.add_node("flip_coin", flip_coin)
builder.add_node("heads_action", heads_action)
builder.add_node("tails_action", tails_action)

# Define edges
builder.add_edge(START, "initialize_state")
builder.add_edge("initialize_state", "flip_coin")

# Edges from the action nodes to the END
builder.add_edge("heads_action", END)
builder.add_edge("tails_action", END)

# Compile the graph
graph = builder.compile()

In [ ]:
graph

In [ ]:
# Run the Graph multiple times
final_state_1 = graph.invoke({})
final_state_2 = graph.invoke({})
final_state_3 = graph.invoke({})
print(f"{final_state_1=}\n{final_state_2=}\n{final_state_3=}")

In [ ]:
from langgraph.graph import StateGraph, MessagesState
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

# class State(TypedDict):
#     messages: Annotated[list, add_messages]
builder = StateGraph(MessagesState) # view source

load_dotenv()
llm = init_chat_model("claude-sonnet-4-5-20250929")

def chatbot(state: MessagesState):
    return {"messages": [llm.invoke(state["messages"])]}

builder.add_node("chatbot", chatbot)
builder.set_entry_point("chatbot")
builder.set_finish_point("chatbot")

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [ ]:
graph

In [ ]:
config = {"configurable": {"thread_id": "1"}}
final_state_1 = graph.invoke({"messages": ["Hi, my name is Huy."]}, config)

In [ ]:
for m in final_state_1["messages"]:
    m.pretty_print()

In [ ]:
final_state_2 = graph.invoke({"messages": ["do you remember my name?"]}, config)

In [ ]:
for m in final_state_2["messages"]:
    m.pretty_print()

In [ ]:

final_snapshot = graph.get_state(config)
final_snapshot # check . options

In [ ]:
for snapshot in graph.get_state_history(config):
  print(snapshot.values)
  print(snapshot.next)
  print("-"*20)

In [ ]:
from typing import Annotated

from langchain_core.tools import tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt

In [ ]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()
llm = init_chat_model("claude-sonnet-4-5-20250929")

builder = StateGraph(MessagesState)

@tool
def human_assistance(query: str) -> str:
    """Useful for when you need human assistance."""
    human_response = interrupt({"query": query})
    return human_response["data"]

tools = [human_assistance]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: MessagesState):
    message = llm_with_tools.invoke(state["messages"])
    assert(len(message.tool_calls) <= 1)
    return {"messages": [message]}

builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
builder.add_node("tools", tool_node)

builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

builder.add_edge("tools", "chatbot")
builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [ ]:
graph

In [ ]:
user_input = "I need some expert guidance for building an AI agent. Could you request assistance for me?"
config = {"configurable": {"thread_id": "1"}}

final_state = graph.invoke({"messages": [user_input]}, config)
for m in final_state["messages"]:
    m.pretty_print()

In [ ]:
human_response = """
    We, the experts are here to help! check out Machine Learning with Hamza on youtube for guidance.
"""

human_command = Command(resume={"data": human_response})

final_state = graph.invoke(human_command, config)

In [ ]:
for m in final_state["messages"]:
    m.pretty_print()

In [ ]:
from langgraph.types import Send
from typing import Annotated
from operator import add

In [ ]:
class OverallState(TypedDict):
    numbers: list[int]
    squared: Annotated[list[str], add]

def continue_to_squaring(state: OverallState):
    return [Send("calculate_square", {"number": n}) for n in state['numbers']]

builder = StateGraph(OverallState)

builder.add_node("calculate_square", lambda state: {"squared": [state["number"]**2]})

builder.add_conditional_edges(START, continue_to_squaring)
builder.add_edge("calculate_square", END)

graph = builder.compile()

In [ ]:
graph

In [ ]:
# Invoking with list of integers
graph.invoke({"numbers": [1,2,3,4]})

In [ ]:
# Define reducer for the 'index' key
def increment_reducer(current_value: int, new_value_from_node: any) -> int:
    return current_value + 1

# Define the Graph State with an 'index'
class LoopState(TypedDict):
    index: Annotated[int, increment_reducer]  # This will track our loop's progress
    output_data: str # Example of other data

# Define a simple node
def loop_iteration_node(state: LoopState) -> LoopState:
    print(f"Processing step {state['index']}")
    return {"index": 0, "output_data": f"Processed at step {state['index']}"}

def should_continue(state: LoopState) -> bool:
    if state["index"] <= 30:
      return "continue"
    return "end"

# Build the Graph with the custom reducer
builder = StateGraph(LoopState)

# Add the node to the workflow
builder.add_node("iterate", loop_iteration_node)

# Add the edges
builder.set_entry_point("iterate")
builder.add_conditional_edges("iterate", should_continue, {"continue": "iterate", "end":END})

# Compile the graph
graph = builder.compile()

In [ ]:
graph

In [ ]:
# Invoke the graph and observe the 'index' increment
initial_state = {"index": 0, "output_data": "start"}

final_state = graph.invoke(initial_state)

In [ ]:
final_state = graph.invoke(initial_state, {"recursion_limit": 50})

In [ ]:
import json
import random
from typing import List, Any, Annotated
from typing_extensions import TypedDict
from langchain_core.messages import ToolMessage, SystemMessage, AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, END

In [ ]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()
llm = init_chat_model("claude-sonnet-4-5-20250929")

@tool
def human_assistance(query: str) -> str:
    """
    This tool is useful when human assistance is requested.
    """
    human_response = interrupt({"query": query})
    return human_response["data"]

@tool
def faq_responses(query: str) -> str:
    """
    This tool is useful to answer frequently asked questions about the company.
    """
    fqa_response = random.choice(["The company was founded in 2025.","No answer found in database"])
    return fqa_response


tools = [human_assistance, faq_responses]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

tools_by_name = {tool.name: tool for tool in tools}

# Define our tool node
def tool_node(state: AgentState):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}


# Define the node that calls the model
def call_model(state: AgentState):
    system_prompt = SystemMessage(
        """
        You are a customer-service assistant.
        You have tools that give you access to a FAQ database and human assistnace.
        When asked about the company, always check with the FAQ database first, and if no answer or unrelated is found, use human assistance.
        """
    )
    response = llm_with_tools.invoke([system_prompt] + state["messages"])
    return {"messages": [response]}


# Define the conditional edge function
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    return "continue"

In [ ]:
# Define a new graph
builder = StateGraph(AgentState)

# Add nodes
builder.add_node("agent", call_model)
builder.add_node("tools", tool_node)

# Add edges (ReAct loop)
builder.set_entry_point("agent")
builder.add_conditional_edges("agent", should_continue, {"continue": "tools","end": END})
builder.add_edge("tools", "agent")

# Now we can compile and visualize our graph
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [ ]:
graph

In [ ]:
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": ["When was the company founded"]}
final_state = graph.invoke(inputs, config)

In [ ]:
for m in final_state["messages"]:
  m.pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "3"}}
inputs = {"messages": ["Who's the company founder?"]}
final_state = graph.invoke(inputs, config)

In [ ]:
for m in final_state["messages"]:
  m.pretty_print()

In [ ]:
human_response = "The company founder is Hamza"

human_command = Command(resume={"data": human_response})

final_state = graph.invoke(human_command, config)

In [ ]:
for m in final_state["messages"]:
  m.pretty_print()